# COLWRIT R4B The Japanese American Internment and its Legacy


---

### Professor Patricia Steenland

This notebook will explore data from the camps and provide context and techniques to analize the forced relocation of Japanese Americans during the 1940s.

*Estimated Time: 90 minutes*

---

### Topics Covered
1. [Section 1: The Jupyter Notebook](#Section-1:-The-Jupyter-Notebook)
2. [Section 2: Data Exploration and Visualization](#Section-2:-Data-Exploration-and-Visualization)
3. [Section 3: Assembly Centers and Internment Centers](#Section-3:-Assembly-Centers-and-Internment-Centers)
4. [Section 4: Mapping and Movement](#Section-4:-Mapping-and-Movement)


---

## Context <a id='data'></a>

In this course, you've been studying the consequences of Japanese American internment. Through this data and subsequent analysis, you'll be able to visualize the forced movement of Japanese Americans, from the west coast, to scattered internment camps, and eventually to cities throughout the country. First, we need to learn how to use this notebook format, which is called a **Jupyter Notebook.**




## Section 1: The Jupyter Notebook

First of all, note that this page is divided into what are called *cells*. You can navigate cells by clicking on them or by using the up and down arrows. Cells will be highlighted as you navigate them.

### Text cells

Text cells (like this one) can be edited by double-clicking on them. They're written in a simple format called [Markdown](http://daringfireball.net/projects/markdown/syntax) to add formatting and section headings.  You don't need to learn Markdown, but know the difference between Text Cells and Code Cells.

### Code cells
Other cells contain code in the Python 3 language. Don't worry -- we'll show you everything you need to know to succeed in this part of the class. 

The fundamental building block of Python code is an **expression**. Cells can contain multiple lines with multiple expressions.  We'll explain what exactly we mean by "expressions" in just a moment: first, let's learn how to "run" cells.

### Running cells

"Running a cell" is equivalent to pressing "Enter" on a calculator once you've typed in the expression you want to evaluate: it produces an **output**. When you run a text cell, it outputs clean, organized writing. When you run a code cell, it **computes** all of the expressions you want to evaluate, and can **output** the result of the computation.

<p></p>

<div class="alert alert-info">
To run the code in a code cell, first click on that cell to activate it.  It'll be highlighted with a little green or blue rectangle.  Next, you can either press the <code><b>▶|</b> Run </code> button above or press <b><code>Shift + Return</code></b> or <b><code>Shift + Enter</code></b>. This will run the current cell and select the next one.
</div>

Text cells are useful for taking notes and keeping your notebook organized, but your data analysis will be done in code cells. We will focus on code cells for the rest of the class.



In [ ]:
print("Hello world!")

In [ ]:
#This is a comment. It is put in code cells as a description or instructions and does not affect the code.

In [ ]:
# Just run this cell
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import plotly.express as px
pop_by_month = pd.read_csv("data/CampPopulationsByMonth.csv", error_bad_lines = False, thousands = ',')
population1940_1945 = pd.read_csv("data/JapaneseAmericanPopulation_1940_1945_LL.csv", error_bad_lines = False)
relocations_cities = pd.read_csv("data/RelocationDestinations_Cities_LL.csv", error_bad_lines = False)
assembly = pd.read_csv('data/BehindBarbedWire_StoryMap_AssemblyCentersMap_Data.csv', error_bad_lines = False)

---

## The Data <a id='data'></a>

In this notebook, you'll be working with a dataset that was manually digitized from tables in The Evacuated People: A Quantitative Description, a report published by the War Relocation Authority in 1946. The tables required manual transcription because the results from automatic scraping contained too many errors. Thus, there may still be some human error. The datasets included contain the populations of each camp by month, the relocation destinations of those who were incarcerated, and the Japanese American population in America before and throughout the war. Take a look at the full dataset <a href = https://data.world/infinitecoop/japanese-internment-camps/> here</a>.

The second dataset we're working with is collected by the Library of Congress and comes from newspapers that were produced by Japanese-American internees while they lived in the camps. You can access the data and read about the newspapers <a href = https://tinyurl.com/y4g5kq77> here</a>.
<img src="images/Posted_Japanese_American_Exclusion_Order.png" width="450">
This image is available from the United States Library of Congress's Prints and Photographs division.

The **pop_by_month** table has the population of each of the ten camps at the start of every month.

In [ ]:
#Loading and Reading the Data
pop_by_month['Date']= pd.to_datetime(pop_by_month['Date']) 
camps = pd.read_csv('data/BehindBarbedWire_StoryMap_InternmentCampLocationsMap_Data.csv', error_bad_lines = False)
camps["Maximum Population"] = camps["Maximum Population"].str.replace(',', '')
camps['Maximum Population'] = camps['Maximum Population'].astype(float)

In [ ]:
pop_by_month.head()

The **camps** table by Behind the Barbed Wire has a list of the internment camps, their locations-- city, name, latitude, and longitude-- the dates of when they opened and closed, and their maximum populations.

In [ ]:
camps

The **assembly** table by Behind the Barbed Wire has the locations of each assembly center-- their city, state, latitude, and longitude--, as well as the number of people that were processed through each one.

In [ ]:
assembly = assembly.dropna(subset=['Latitude', 'Longitude']).dropna(axis = 1)
assembly['Number of People'] = assembly['Number of People'].astype(float)
assembly

The **population1940_1945** table has a list of counties that Japanese Americans lived in in 1940 and 1945. A separate column calculates the percent that returned to the county that they were from.

In [ ]:
population1940_1945.head()

# Section 2: Data Exploration and Visualization

Sometimes numbers don't always add up. We have two separate tables with data about the internment camps, collected by two separate organizations. Say we try to compare the maximum population of the Manzanar camps from both tables.

In [ ]:
pop_by_month_max = pop_by_month['Manzanar'].max()
camps_max = int(camps[camps['Internment Camp Name'] == 'Manzanar Relocation Center']['Maximum Population'])
print('Camps Data:',  camps_max)
print('Popuation by Month Data:' ,  pop_by_month_max)
#Are the two numbers equal?

The two numbers don't equal each other! The **pop_by_month** table says that the maximum population of Manzanar was 10,256, while the camps table says that the maximum was 10,046 people. While this isn't a huge difference, it's important to remember that there can be error in your datasets, and not to take any one dataset as complete and total fact. 

Now we can look at the data and start to see beyond the numbers. **This over-laid line plot compares the change in populations of the camps by month.** 

In [ ]:
pop_by_month2 = pop_by_month.drop("Total", axis = 1)
melted = pd.melt(pop_by_month2, id_vars = ["Date"], value_vars = pop_by_month2.columns[1:], var_name = 'Camp', value_name = "Population")
fig = px.line(melted, x = 'Date', y = 'Population', color = 'Camp', title = 'Camp Populations by Month')
fig.show();

We can see from this histogram that most people did not return to where they originally lived because the bin that is at 0 percent returned has the highest percentage (the bar is the highest closest to 0). 

In [ ]:
sns.distplot(population1940_1945['% returned'])
plt.title("Percent Returned to Original County");


**Question:** Visualizations allow us to picture how the numbers change and find abnormalities in the data set. What are some abnormalities that you see in the first plot? What are some possible explanations for these abnormalities?


**Answer:** YOUR ANSWER HERE

---

## Section 3: Assembly Centers and Internment Centers<a id='section 2'></a>

After Executive Order 9066 was signed, the army was authorized to remove civilians from designated military exclusion zones spanning Washington, Oregon, California, and Arizona. Assembly centers were created to funnel Japanese Americans into internment camps. Through these maps, you can see how far people were forced to move from one location to another. Mapping data is a useful tool to visualize locations and provide context when we are given longitude and latitude data! 
<img src="images/Luggage_Japanese_American_internment.png" width="400">
The luggage of the Japanese Americans who have arrived at a reception center.
This image is available from the United States Library of Congress's Prints and Photographs division.

### Assembly Centers
Japanese Americans were transported from assembly centers to internment camps across the country and we can use mapping techniques to visualize the movement of Japanese Americans from their original homes, to assembly centers, to internment camps, and then to various locations afterwards. To zoom in and out on the map, you can press the + and - buttons or scroll up and down. Also, if you click a blue marker, the assembly center name with show up!

In [ ]:
m = folium.Map(location=[36.733300, -100.766700], zoom_start=4)
tooltip = 'Click me!'
for i in range(0,len(assembly)):
    folium.Marker(
      location=(assembly.iloc[i]['Latitude'], assembly.iloc[i]['Longitude']),
      popup=assembly.iloc[i]['Location'],
      ).add_to(m)
m

### Internment Camps


In [ ]:
m = folium.Map(location=[36.733300, -100.766700], zoom_start=4)
tooltip = 'Click me!'
for i in range(0,len(camps)):
    folium.Marker(
      location=(camps.iloc[i]['Latitude'], camps.iloc[i]['Longitude']),
      popup=camps.iloc[i]['Internment Camp Name'],
      ).add_to(m)
m

## Section 4: Mapping and Movement<a id='section 3'></a>

One of the consequences of the Japanese internment camps was the relocation after World War 2. Many of the Japanese Americans that lived on the West Coast of the United States did not return to their previous homes since land holdings and possessions were lost during the process. This map shows the populations of each county at the start of World War II, just before the Executive Order 9066 in 1942. Japanese Americans were largely concentrated on the West Coast, with the largest population in Los Angeles.

<img src="images/Hayward_Friends_say_goodbye.png" width="400">
Neighbors in Hayward, California saying goodbye.
This image is available from the United States Library of Congress's Prints and Photographs division.

In [ ]:
m = folium.Map(location=[36.733300, -100.766700], zoom_start=4)
tooltip = 'Click me!'
for i in range(0,len(population1940_1945)):
    folium.Circle(
      location = (population1940_1945.iloc[i]['Latitude'], population1940_1945.iloc[i]['Longitude']),
      popup = population1940_1945.iloc[i]['County'],
      radius = float(population1940_1945.iloc[i]['1940']) *12,
      color = 'blue',
      fill = True,
      fill_color = 'crimson').add_to(m)
m

This table shows the amount of Japanese Americans in each county after the war. This data was self-reported by evacuees.

In [ ]:
reloc_cities = pd.read_csv('data/RelocationDestinations_Cities_LL.csv', error_bad_lines = False)
reloc_cities['People'] = reloc_cities['People'].astype(float)
reloc_cities.head()

This map shows the cities that people relocated after the war. It allows us to visualize the extent people had to relocate from their original homes which were mostly on the west coast of the U.S. We can see a large population moved to the midwest (Chicago area) as well as the east coast, Canada, and even Mexico City. 

In [ ]:
m = folium.Map(location=[36.733300, -100.766700], zoom_start=4)
tooltip = 'Click me!'
for i in range(0,len(reloc_cities)):
    folium.Circle(
      location=(reloc_cities.iloc[i]['Latitude'], reloc_cities.iloc[i]['Longitude']),
      popup=reloc_cities.iloc[i]['City'],
      radius=reloc_cities.iloc[i]['People'] * 12,
      color='blue',
      fill=True,
      fill_color='red').add_to(m)
m

Below is the table comparing the pre- and post-internment population sizes for West Coast states. The table has been separated into the three states for easier readability. Look through the population size values and look for trends that may be interesting to study in a future research project. 

In [ ]:
population1940_1945[population1940_1945['State'] == 'CA']

In [ ]:
population1940_1945[population1940_1945['State'] == 'OR']

In [ ]:
population1940_1945[population1940_1945['State'] == 'WA']


**Question:** Based on the tables and maps above, identify a research question that could be pursued further using other types of data than we have included here, such as personal narratives or news articles.


**Answer:** YOUR ANSWER HERE

---

**Make sure that you've answered both questions.**

You are finished with this notebook! Please run the following cell to generate a download link for your submission file to submit on bCourses. ***It is very likely that this download link will not work. If the download link does not work, please use the alternate download method described below.***

Alternate download instructions:
- open a new tab and go to https://datahub.berkeley.edu
- go to the `colwrit-r4b` folder
- click the box next to `Japanese-Internment-notebook.pdf`
- click the "Download" link below the menu bar

**Check the PDF before submitting and make sure all of your answers are shown.**

In [ ]:
!pip install gsexport -q
import gsExport
gsExport.generateSubmission("Japanese-Internment-notebook.ipynb")

---
Notebook developed by: Alleanna Clark, Aishah Mahmud, Monica Wilkinson

Data Science Modules: http://data.berkeley.edu/education/modules
